# LCEL

https://python.langchain.com/docs/expression_language/

In [9]:
# get api key
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Chapter 7: OutputParsers and Memory

### 7.1 Parsers

In [2]:
# 7.1.1
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai.llms import OpenAI

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt_template = """
Suggest some names for my {subject} startup. \n
{format_instructions}
"""

prompt = PromptTemplate.from_template(prompt_template)

llm = OpenAI()

chain = prompt | llm | output_parser

print(
    chain.invoke(
        {
            "format_instructions": format_instructions,
            "subject": "Mobile",
        }
    )
)

['1. SwiftTech\n2. MobileGenius\n3. PocketPal\n4. TechMobility\n5. Cloud9Mobile\n6. DigiHive\n7. MobileMaven\n8. AppTastic\n9. OnTheGoTech\n10. TouchTech\n11. MobileEdge\n12. PocketRocket\n13. TechSavvy\n14. MobileMatrix\n15. CloudMinds\n16. AppXcelerate\n17. PocketTech\n18. OnTheMoveTech\n19. MobileSphere\n20. TechTrailblazers']


In [3]:
# 7.1.2 Custom OutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain_openai.llms import OpenAI
from langchain.output_parsers import OutputFixingParser
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough

response_schemas = [
    ResponseSchema(
        name="Monument",
        description="The monument mentioned in the answer.",
    ),
    ResponseSchema(
        name="city", description="the city in which this monument is present."
    ),
    ResponseSchema(
        name="architect",
        description="the architect of the monument.",
    ),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Monument": string  // The monument mentioned in the answer.
	"city": string  // the city in which this monument is present.
	"architect": string  // the architect of the monument.
}
```


In [4]:
prompt_template = """
    answer the users question as best as possible. \n
    {format_instructions}\n {question}
"""

prompt = PromptTemplate.from_template(prompt_template)

llm = OpenAI()

chain = prompt | llm | output_parser

print(
    chain.invoke(
        {
            "format_instructions": format_instructions,
            "question": "Choose a random Indian state and pen down some cities",
        }
    )
)

OutputParserException: Got invalid JSON object. Error: Expecting property name enclosed in double quotes: line 2 column 27 (char 28)

In [ ]:
# 7.13

from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from typing import List
from langchain_openai import OpenAI


class State(BaseModel):
    state: str = Field(description="name of an Indian State")
    cities: List[str] = Field(description="Indian cities in that state")


parser = PydanticOutputParser(pydantic_object=State)

format_instructions = parser.get_format_instructions()

template = """answer the users question as best as possible and generate output. \n
{format_instructions} \n{questions}
"""

prompt = PromptTemplate.from_template(template)
llm = OpenAI(temperature=0, api_key=OPENAI_API_KEY)
chain = prompt | llm | parser

chain.invoke(
    {
        "questions": "State is Andhra Pradesh",
        "format_instructions": format_instructions,
    }
)

State(state='Andhra Pradesh', cities=['Visakhapatnam', 'Vijayawada', 'Guntur', 'Nellore', 'Kurnool', 'Rajahmundry', 'Kakinada', 'Tirupati', 'Anantapur', 'Kadapa'])

### 7.2 Memory

In [ ]:
# coversational buffer memory
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import os

In [ ]:
template = """You are a chatbot having a conversation with a human.
{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate.from_template(template)
memory = ConversationBufferMemory(memory_key="chat_history")
llm = OpenAI(api_key=OPENAI_API_KEY, temperature=0)

chain = LLMChain(
    prompt=prompt,
    memory=memory,
    llm=llm,
    verbose=True,
)

chain.invoke("hi there, my name is Raghu")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Human: hi there, my name is Raghu
Chatbot:

> Finished chain.


{'human_input': 'hi there, my name is Raghu',
 'chat_history': '',
 'text': " Hello Raghu, it's nice to meet you. I am a chatbot designed to have conversations with humans. How can I assist you today?"}

In [ ]:
chain.invoke("write a song about me")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.
Human: hi there, my name is Raghu
AI:  Hello Raghu, it's nice to meet you. I am a chatbot designed to have conversations with humans. How can I assist you today?
Human: write a song about me
Chatbot:

> Finished chain.


{'human_input': 'write a song about me',
 'chat_history': "Human: hi there, my name is Raghu\nAI:  Hello Raghu, it's nice to meet you. I am a chatbot designed to have conversations with humans. How can I assist you today?",
 'text': " Sure, Raghu, here's a little song I came up with just for you:\n\nVerse 1:\nRaghu, oh Raghu, with a heart of gold\nYour smile shines bright, never growing old\nYou light up the room with your infectious laugh\nAnd your kindness and warmth, they're hard to surpass\n\nChorus:\nOh Raghu, you're one of a kind\nA true gem, so hard to find\nYour presence brings joy, wherever you go\nYou're a shining star, don't you know?\n\nVerse 2:\nWith your passion and drive, you reach for the stars\nAnd your determination, it knows no bars\nYou inspire those around you, to be their best\nAnd with you by their side, they feel truly blessed\n\nChorus:\nOh Raghu, you're one of a kind\nA true gem, so hard to find\nYour presence brings joy, wherever you go\nYou're a shining star

In [ ]:
chain.invoke("what is the song about?")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.
Human: hi there, my name is Raghu
AI:  Hello Raghu, it's nice to meet you. I am a chatbot designed to have conversations with humans. How can I assist you today?
Human: write a song about me
AI:  Sure, Raghu, here's a little song I came up with just for you:

Verse 1:
Raghu, oh Raghu, with a heart of gold
Your smile shines bright, never growing old
You light up the room with your infectious laugh
And your kindness and warmth, they're hard to surpass

Chorus:
Oh Raghu, you're one of a kind
A true gem, so hard to find
Your presence brings joy, wherever you go
You're a shining star, don't you know?

Verse 2:
With your passion and drive, you reach for the stars
And your determination, it knows no bars
You inspire those around you, to be their best
And with you by their side, they feel truly blessed

Chorus:
Oh Raghu, you're one of a kind
A true gem, so hard to find
Your presenc

{'human_input': 'what is the song about?',
 'chat_history': "Human: hi there, my name is Raghu\nAI:  Hello Raghu, it's nice to meet you. I am a chatbot designed to have conversations with humans. How can I assist you today?\nHuman: write a song about me\nAI:  Sure, Raghu, here's a little song I came up with just for you:\n\nVerse 1:\nRaghu, oh Raghu, with a heart of gold\nYour smile shines bright, never growing old\nYou light up the room with your infectious laugh\nAnd your kindness and warmth, they're hard to surpass\n\nChorus:\nOh Raghu, you're one of a kind\nA true gem, so hard to find\nYour presence brings joy, wherever you go\nYou're a shining star, don't you know?\n\nVerse 2:\nWith your passion and drive, you reach for the stars\nAnd your determination, it knows no bars\nYou inspire those around you, to be their best\nAnd with you by their side, they feel truly blessed\n\nChorus:\nOh Raghu, you're one of a kind\nA true gem, so hard to find\nYour presence brings joy, wherever you 

In [ ]:
memory.load_memory_variables({})

{'chat_history': "Human: hi there, my name is Raghu\nAI:  Hello Raghu, it's nice to meet you. I am a chatbot designed to have conversations with humans. How can I assist you today?\nHuman: write a song about me\nAI:  Sure, Raghu, here's a little song I came up with just for you:\n\nVerse 1:\nRaghu, oh Raghu, with a heart of gold\nYour smile shines bright, never growing old\nYou light up the room with your infectious laugh\nAnd your kindness and warmth, they're hard to surpass\n\nChorus:\nOh Raghu, you're one of a kind\nA true gem, so hard to find\nYour presence brings joy, wherever you go\nYou're a shining star, don't you know?\n\nVerse 2:\nWith your passion and drive, you reach for the stars\nAnd your determination, it knows no bars\nYou inspire those around you, to be their best\nAnd with you by their side, they feel truly blessed\n\nChorus:\nOh Raghu, you're one of a kind\nA true gem, so hard to find\nYour presence brings joy, wherever you go\nYou're a shining star, don't you know?\

#### 7.2.2 ConversationSummaryMemory

In [ ]:
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory

In [ ]:
memory = ConversationSummaryMemory(
    llm=OpenAI(api_key=OPENAI_API_KEY, temperature=0))

memory.save_context(
    {"input": "hi, I am Raghu"}, {"output": "hello Raghu? how can I help"}
)

In [ ]:
llm = OpenAI(temperature=0)
chat = ConversationChain(llm=llm, memory=memory, verbose=True)

In [ ]:
chat.invoke(input="what is GPT4o model?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as Raghu and the AI responds by asking how it can help.
Human: what is GPT4o model?
AI:

> Finished chain.


{'input': 'what is GPT4o model?',
 'history': '\nThe human introduces themselves as Raghu and the AI responds by asking how it can help.',
 'response': ' GPT4o is a natural language processing model developed by OpenAI. It is the successor to GPT-3 and is designed to generate human-like text responses. It has 175 billion parameters, making it one of the largest language models in existence. It is trained on a diverse range of internet text data and can perform a variety of tasks such as language translation, text completion, and question-answering. Is there anything specific you would like to know about GPT4o?'}

In [ ]:
chat.invoke(input="what is the difference b/w GPT4 and GPT4o")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as Raghu and the AI responds by asking how it can help. The human then asks about GPT4o, a natural language processing model developed by OpenAI with 175 billion parameters. The AI explains that it is trained on internet text data and can perform various tasks. The AI asks if there is anything specific the human would like to know about GPT4o.
Human: what is the difference b/w GPT4 and GPT4o
AI:

> Finished chain.


{'input': 'what is the difference b/w GPT4 and GPT4o',
 'history': '\nThe human introduces themselves as Raghu and the AI responds by asking how it can help. The human then asks about GPT4o, a natural language processing model developed by OpenAI with 175 billion parameters. The AI explains that it is trained on internet text data and can perform various tasks. The AI asks if there is anything specific the human would like to know about GPT4o.',
 'response': ' GPT4o is an improved version of GPT4, with a larger number of parameters and improved training methods. GPT4o has 175 billion parameters, while GPT4 has 175 billion parameters. This allows GPT4o to have a better understanding of natural language and perform more complex tasks. Additionally, GPT4o is trained on a larger and more diverse dataset, including internet text data, which helps it to better understand and generate human-like language.'}

In [ ]:
messages = memory.chat_memory.messages
messages

[HumanMessage(content='hi, I am Raghu'),
 AIMessage(content='hello Raghu? how can I help'),
 HumanMessage(content='what is GPT4o model?'),
 AIMessage(content=' GPT4o is a natural language processing model developed by OpenAI. It is the successor to GPT-3 and is designed to generate human-like text responses. It has 175 billion parameters, making it one of the largest language models in existence. It is trained on a diverse range of internet text data and can perform a variety of tasks such as language translation, text completion, and question-answering. Is there anything specific you would like to know about GPT4o?'),
 HumanMessage(content='what is the difference b/w GPT4 and GPT4o'),
 AIMessage(content=' GPT4o is an improved version of GPT4, with a larger number of parameters and improved training methods. GPT4o has 175 billion parameters, while GPT4 has 175 billion parameters. This allows GPT4o to have a better understanding of natural language and perform more complex tasks. Additi

## Chapter 8. Callbacks

In [ ]:
# 8.2
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI

# from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os

In [ ]:
handler = StdOutCallbackHandler()
llm = OpenAI(api_key=OPENAI_API_KEY, temperature=0)

prompt = PromptTemplate.from_template("What is the capital of {country}?")
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(country="Canada")

'\n\nThe capital of Canada is Ottawa.'

In [ ]:
chain.run(country="Canada", callbacks=[handler])



> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of Canada?

> Finished chain.


'\n\nThe capital of Canada is Ottawa.'

In [11]:
# 8.3 file handler
from langchain.callbacks import FileCallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from loguru import logger

In [8]:
logfile = "../files/logs/output.log"

In [12]:
logger.add(logfile, colorize=True, enqueue=True)
handler = FileCallbackHandler(logfile)

llm = OpenAI(temperature=0)
prompt = PromptTemplate.from_template("What is the capital of {country}? ")
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])

answer = chain.invoke({"country": "United States"})
answer

{'country': 'United States',
 'text': '\n\nThe capital of United States is Washington, D.C.'}

In [13]:
logger.info(answer)

2024-05-28 15:26:08.280 | INFO     | __main__:<module>:1 - {'country': 'United States', 'text': '\n\nThe capital of United States is Washington, D.C.'}


In [7]:
# 8.4 Custom callbacks
from langchain.callbacks.base import BaseCallbackHandler


class MyHandler(BaseCallbackHandler):
    def on_llm_start(
        self,
        serialized: "Dict[str, Any]",
        prompts: "List[str]",
        *,
        run_id: "UUID",
        parent_run_id: "Optional[UUID]" = None,
        tags: "Optional[List[str]]" = None,
        metadata: "Optional[Dict[str, Any]]" = None,
        **kwargs: "Any"
    ):
        print("We are starting of with this prompt -->", prompts)

    def on_llm_end(
        self,
        response: "LLMResult",
        *,
        run_id: "UUID",
        parent_run_id: "Optional[UUID]" = None,
        **kwargs: "Any"
    ):
        print("The LLM is done", response)

In [12]:
handler = MyHandler()
llm = OpenAI()
prompt = PromptTemplate.from_template("What is the capital of {country}?")
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(country="Argentina", callbacks=[handler])

/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_AI/dev_proj_AI/pract-langchain/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


We are starting of with this prompt --> ['What is the capital of Argentina?']
The LLM is done generations=[[Generation(text='\n\nThe capital of Argentina is Buenos Aires.', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'total_tokens': 16, 'completion_tokens': 9, 'prompt_tokens': 7}, 'model_name': 'gpt-3.5-turbo-instruct'} run=None


'\n\nThe capital of Argentina is Buenos Aires.'

## Chapter 9: RAG Framework and Vector Databases